In [15]:
%pip install -qU -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


Load Env Variables and Secrets

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('../../../azure.env')
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-06-01"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = 'gpt-4o-mini'
os.environ["AZURE_OPENAI_MODEL_VERSION"] = '2024-06-01'


Import packages

In [5]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import FireCrawlLoader

Initialize the Model

In [3]:
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    model_version=os.environ['AZURE_OPENAI_MODEL_VERSION']
)

In [4]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    # azure_endpoint="https://<your-endpoint>.openai.azure.com/", If not provided, will read env variable AZURE_OPENAI_ENDPOINT
    # api_key=... # Can provide an API key directly. If missing read env variable AZURE_OPENAI_API_KEY
    # openai_api_version=..., # If not provided, will read env variable AZURE_OPENAI_API_VERSION
)

In [7]:
# Step 1: Scrape the content from apple.com using WebBaseLoader
# WebBaseLoader loads web pages and extracts their content
urls = "https://en.wikipedia.org/wiki/Romeo_%2B_Juliet"

# Create a loader for web content
loader = FireCrawlLoader(url=urls, mode="scrape")
documents = loader.load()

In [12]:
documents

[Document(metadata={'title': 'Romeo + Juliet - Wikipedia', 'language': 'en', 'robots': 'max-image-preview:standard', 'ogTitle': 'Romeo + Juliet - Wikipedia', 'ogImage': 'https://upload.wikimedia.org/wikipedia/en/b/b4/William_shakespeares_romeo_and_juliet_movie_poster.jpg', 'ogLocaleAlternate': [], 'sourceURL': 'https://en.wikipedia.org/wiki/Romeo_%2B_Juliet', 'statusCode': 200}, page_content='[Jump to content](#bodyContent)\n\n[![Page semi-protected](https://en.wikipedia.org/upload.wikimedia.org/wikipedia/en/thumb/1/1b/Semi-protection-shackle.svg/20px-Semi-protection-shackle.svg.png)](/wiki/Wikipedia:Protection_policy#semi "This article is semi-protected.")\n\nFrom Wikipedia, the free encyclopedia\n\n1996 film directed by Baz Luhrmann\n\nThis article is about the film. For Peter Martins\' 2007 ballet, see [_Romeo + Juliet_ (ballet)](/wiki/Romeo_%2B_Juliet_(ballet) "Romeo + Juliet (ballet)").\n\n"William Shakespeare\'s Romeo and Juliet" redirects here. For the original play by William S

In [9]:
# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Display information about the split documents
print("\n--- Document Chunks Information ---")
print(f"Number of document chunks: {len(docs)}")

# Create the vector store and persist it automatically
print("\n--- Creating vector store ---")
vectorstore = InMemoryVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
)
print("\n--- Finished creating vector store ---")


--- Document Chunks Information ---
Number of document chunks: 107

--- Creating vector store ---

--- Finished creating vector store ---


In [10]:
retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3},
        )

In [13]:
# Define the user's question
query = "How did Juliet die?"

# Retrieve relevant documents based on the query
relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"Source: {doc.metadata.get('title', 'Unknown')}\n")


--- Relevant Documents ---
Document 1:
Romeo enters the church where Juliet lies and consumes the poison just as Juliet wakes up. Distraught over Romeo’s death, Juliet picks up his gun and shoots herself in the head, falling down beside his lifeless body. Romeo's body is being taken inside an ambulance with a crowd of spectators and reporters observing the incident from behind the police line, when the parents of both Romeo and Juliet arrive on the scene. Captain Prince approaches their fathers, and berates them both for the deaths of their children that their foolish brawl has caused.

## Cast

Main article: [Characters in Romeo and Juliet](/wiki/Characters_in_Romeo_and_Juliet "Characters in Romeo and Juliet")

Source: Romeo + Juliet - Wikipedia

Document 2:
The next morning, Gloria informs Juliet that she is to marry Paris, and when Juliet refuses, Fulgencio physically assaults her and threatens to disown her. Juliet runs away and seeks out Father Laurence, imploring him to help her

In [14]:
# Combine the query and the relevant document contents
combined_input = (
    "Here are some documents that might help answer the question: "
    + query
    + "\n\nRelevant Documents:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nPlease provide an answer based only on the provided documents. If the answer is not found in the documents, respond with 'I'm not sure'."
)

In [16]:
# Define the messages for the model
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result.content)


--- Generated Response ---
Content only:
Juliet died by shooting herself in the head with Romeo's gun after she woke up to find Romeo dead beside her.
